In [2]:
!git clone https://github.com/neural-maze/agentic_patterns.git

fatal: destination path 'agentic_patterns' already exists and is not an empty directory.


In [3]:
%cd agentic_patterns

/content/agentic_patterns


In [4]:
!pip install groq==0.10.0 jupyter==1.0.0 python-dotenv==1.0.1 colorama==0.4.6 types-colorama==0.4.15.20240311 graphviz==0.20.3 httpx==0.27.2 json

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


# Tool Pattern


---

As you may already know, the information stored in LLM weights is (usually) 𝐧𝐨𝐭 𝐞𝐧𝐨𝐮𝐠𝐡 to give accurate and insightful answers to our questions.

That's why we need to provide the LLM with ways to access the outside world. 🌍

In practice, you can build tools for whatever you want (at the end of the day they are just functions the LLM can use), from a tool that let's you access Wikipedia, another to analyse the content of YouTube videos or calculate difficult integrals using Wolfram Alpha.

The second pattern we are going to implement is the **tool pattern**.

In this notebook, you'll learn how **tools** actually work. This is the **second lesson** of the "Agentic Patterns from Scratch" series. Take a look at the first lesson if you haven't!

* [First Lesson: The Reflection Pattern](https://github.com/neural-maze/agentic_patterns/blob/main/notebooks/reflection_pattern.ipynb)

## A simple function

Take a look at this function 👇

In [5]:
import json

def get_current_weather(location: str, unit: str):
	"""
	Get the current weather in a given location

	location (str): The city and state, e.g. Madrid, Barcelona
	unit (str): The unit. It can take two values; "celsius", "fahrenheit"
	"""
	if location == "Madrid":
		return json.dumps({"temperature": 25, "unit": unit})

	else:
		return json.dumps({"temperature": 58, "unit": unit})

Very simple, right? You provide a `location` and a `unit` and it returns the temperature.

In [6]:
get_current_weather(location="Madrid", unit="celsius")

'{"temperature": 25, "unit": "celsius"}'

But the question is:

**How can you make this function available to an LLM?**

An LLM is a type of NLP system, so it expects text as input. But how can we transform this function into text?

## A System Prompt that works

For the LLM to be aware of this function, we need to provide some relevant information about it in the context. **I'm referring to the function name, attributes, description, etc.** Take a look at the following System Prompt.

```xml
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

Here are the available tools:

<tools> {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location location (str): The city and state, e.g. Madrid, Barcelona unit (str): The unit. It can take two values; 'celsius', 'fahrenheit'",
    "parameters": {
        "properties": {
            "location": {
                "type": "string"
            },
            "unit": {
                "type": "string"
            }
        }
    }
}
</tools>
```


As you can see, the LLM enforces the LLM to behave as a `function calling AI model` who, given a list of function signatures inside the <tools></tools> XML tags
will select which one to use. When the model decides a function to use, it will return a json like the following, representing a function call:

```xml
<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>
```


Let's see how it works in practise! 👇

In [7]:
import os
import re
from groq import Groq
from dotenv import load_dotenv
import json

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

MODEL = "llama-3.3-70b-versatile"
GROQ_CLIENT = Groq(api_key='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')

# Define the System Prompt as a constant
TOOL_SYSTEM_PROMPT = """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

Here are the available tools:

<tools> {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location location (str): The city and state, e.g. Madrid, Barcelona unit (str): The unit. It can take two values; 'celsius', 'fahrenheit'",
    "parameters": {
        "properties": {
            "location": {
                "type": "str"
            },
            "unit": {
                "type": "str"
            }
        }
    }
}
</tools>
"""

Let's ask a very simple question: `"What's the current temperature in Madrid, in Celsius?"`

In [8]:
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_SYSTEM_PROMPT
    }
]
agent_chat_history = []

user_msg = {
    "role": "user",
    "content": "What's the current temperature in Madrid, in Celsius?"
}

tool_chat_history.append(user_msg)
agent_chat_history.append(user_msg)

output = GROQ_CLIENT.chat.completions.create(
    messages=tool_chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<tool_call>
{"name": "get_current_weather", "arguments": {"location": "Madrid", "unit": "celsius"}}
</tool_call>


---

**That's an improvement!** We may not have the *proper* answer but, with this information, we can obtain it! How? Well, we just need to:

1. Parse the LLM output. By this I mean deleting the XML tags
2. Load the output as a proper Python dict

The function below does exactly this.

---

In [9]:
def parse_tool_call_str(tool_call_str: str):
    pattern = r'</?tool_call>'
    clean_tags = re.sub(pattern, '', tool_call_str)

    try:
        tool_call_json = json.loads(clean_tags)
        return tool_call_json
    except json.JSONDecodeError:
        return clean_tags
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "There was some error parsing the Tool's output"

In [10]:
parsed_output = parse_tool_call_str(output)
parsed_output

{'name': 'get_current_weather',
 'arguments': {'location': 'Madrid', 'unit': 'celsius'}}

We can simply run the function now, by passing the arguments like this 👇

In [11]:
result = get_current_weather(**parsed_output["arguments"])

In [12]:
result

'{"temperature": 25, "unit": "celsius"}'

**That's it!** A temperature of 25 degrees Celsius.

As you can see, we're dealing with a string, so we can simply add the parsed_output to the `chat_history` so that the LLM knows the information it has to return to the user.

In [13]:
agent_chat_history.append({
    "role": "user",
    "content": f"Observation: {result}"
})

In [14]:
GROQ_CLIENT.chat.completions.create(
    messages=agent_chat_history,
    model=MODEL
).choices[0].message.content

'The current temperature in Madrid is 25°C.'

## Implementing everything the good way

To recap, we have a way for the LLM to generate `tool_calls` that we can use later to *properly* run the functions. But, as you may imagine, there are some pieces missing:

1. We need to automatically transform any function into a description like we saw in the initial system prompt.
2. We need a way to tell the agent that this function is a tool

Let's do it!

### The `tool` decorator

We are going to use the `tool` decorator to transform any Python function into a tool. You can see the implementation [here](https://github.com/neural-maze/agentic_patterns/blob/main/src/agentic_patterns/tool_pattern/tool.py). To test it out, let's make a more complex tool than before. For example, a tool that interacts with [Hacker News](https://news.ycombinator.com/), getting the current top stories.

> Reminder: To automatically generate the function signature for the tool, we need a way to infer the arguments types. For this reason, we need to create the typing annotations.

In [15]:
import json
import requests
from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.tool_pattern.tool_agent import ToolAgent

def fetch_top_hacker_news_stories(top_n: int):
    """
    Fetch the top stories from Hacker News.

    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API.
    Each story contains the title, URL, score, author, and time of submission. The data is fetched
    from the official Firebase Hacker News API, which returns story details in JSON format.

    Args:
        top_n (int): The number of top stories to retrieve.
    """
    top_stories_url = 'https://hacker-news.firebaseio.com/v0/topstories.json'

    try:
        response = requests.get(top_stories_url)
        response.raise_for_status()  # Check for HTTP errors

        # Get the top story IDs
        top_story_ids = response.json()[:top_n]

        top_stories = []

        # For each story ID, fetch the story details
        for story_id in top_story_ids:
            story_url = f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json'
            story_response = requests.get(story_url)
            story_response.raise_for_status()  # Check for HTTP errors
            story_data = story_response.json()

            # Append the story title and URL (or other relevant info) to the list
            top_stories.append({
                'title': story_data.get('title', 'No title'),
                'url': story_data.get('url', 'No URL available'),
            })

        return json.dumps(top_stories)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return []

If we run this Python function, we'll obtain the top HN stories, as you can see below (the top 5 in this case).

In [16]:
json.loads(fetch_top_hacker_news_stories(top_n=5))

[{'title': 'DeepSeek: X2 Speed for WASM with SIMD',
  'url': 'https://simonwillison.net/2025/Jan/27/llamacpp-pr/'},
 {'title': 'DeepSeek improved the Transformer architecture',
  'url': 'https://epoch.ai/gradient-updates/how-has-deepseek-improved-the-transformer-architecture'},
 {'title': 'New Speculative Attacks on Apple CPUs',
  'url': 'https://predictors.fail/'},
 {'title': 'Using UV as Your Shebang Line',
  'url': 'https://akrabat.com/using-uv-as-your-shebang-line/'},
 {'title': 'Maxima in the browser using Embedded Common Lisp on WASM',
  'url': 'https://maxima-on-wasm.pages.dev/'}]

To transform the `fetch_top_hacker_news_stories` function into a Tool, we can use the `tool` decorator.

In [17]:
hn_tool = tool(fetch_top_hacker_news_stories)

The Tool has the following parameters: a `name`, a `fn_signature` and the `fn` (this is the function we are going to call, this case `fetch_top_hacker_news_stories`)

In [18]:
hn_tool.name

'fetch_top_hacker_news_stories'

By default, the tool gets its name from the function name.

In [19]:
json.loads(hn_tool.fn_signature)

{'name': 'fetch_top_hacker_news_stories',
 'description': '\n    Fetch the top stories from Hacker News.\n\n    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API. \n    Each story contains the title, URL, score, author, and time of submission. The data is fetched \n    from the official Firebase Hacker News API, which returns story details in JSON format.\n\n    Args:\n        top_n (int): The number of top stories to retrieve.\n    ',
 'parameters': {'properties': {'top_n': {'type': 'int'}}}}

As you can see, the function signature has been automatically generated. It contains the `name`, a `description` (taken from the docstrings) and the `parameters`, whose types come from the tying annotations. Now that we have a tool, let's run the agent.

### The `ToolAgent`

To create the agent, we just need to pass a list of tools (in this case, just one).

In [23]:
import json
import re

from colorama import Fore
from dotenv import load_dotenv
from groq import Groq

from agentic_patterns.tool_pattern.tool import Tool
from agentic_patterns.tool_pattern.tool import validate_arguments
from agentic_patterns.utils.completions import build_prompt_structure
from agentic_patterns.utils.completions import ChatHistory
from agentic_patterns.utils.completions import completions_create
from agentic_patterns.utils.completions import update_chat_history
from agentic_patterns.utils.extraction import extract_tag_content

load_dotenv()


TOOL_SYSTEM_PROMPT = """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call>
XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>,  "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools:

<tools>
%s
</tools>
"""


class ToolAgent:
    """
    The ToolAgent class represents an agent that can interact with a language model and use tools
    to assist with user queries. It generates function calls based on user input, validates arguments,
    and runs the respective tools.

    Attributes:
        tools (Tool | list[Tool]): A list of tools available to the agent.
        model (str): The model to be used for generating tool calls and responses.
        client (Groq): The Groq client used to interact with the language model.
        tools_dict (dict): A dictionary mapping tool names to their corresponding Tool objects.
    """

    def __init__(
        self,
        tools: Tool | list[Tool],
        model: str = "llama-3.3-70b-versatile",
    ) -> None:
        self.client = Groq(api_key='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')
        self.model = model
        self.tools = tools if isinstance(tools, list) else [tools]
        self.tools_dict = {tool.name: tool for tool in self.tools}

    def add_tool_signatures(self) -> str:
        """
        Collects the function signatures of all available tools.

        Returns:
            str: A concatenated string of all tool function signatures in JSON format.
        """
        return "".join([tool.fn_signature for tool in self.tools])

    def process_tool_calls(self, tool_calls_content: list) -> dict:
        """
        Processes each tool call, validates arguments, executes the tools, and collects results.

        Args:
            tool_calls_content (list): List of strings, each representing a tool call in JSON format.

        Returns:
            dict: A dictionary where the keys are tool call IDs and values are the results from the tools.
        """
        observations = {}
        for tool_call_str in tool_calls_content:
            tool_call = json.loads(tool_call_str)
            tool_name = tool_call["name"]
            tool = self.tools_dict[tool_name]

            print(Fore.GREEN + f"\nUsing Tool: {tool_name}")

            # Validate and execute the tool call
            validated_tool_call = validate_arguments(
                tool_call, json.loads(tool.fn_signature)
            )
            print(Fore.GREEN + f"\nTool call dict: \n{validated_tool_call}")

            result = tool.run(**validated_tool_call["arguments"])
            print(Fore.GREEN + f"\nTool result: \n{result}")

            # Store the result using the tool call ID
            observations[validated_tool_call["id"]] = result

        return observations

    def run(
        self,
        user_msg: str,
    ) -> str:
        """
        Handles the full process of interacting with the language model and executing a tool based on user input.

        Args:
            user_msg (str): The user's message that prompts the tool agent to act.

        Returns:
            str: The final output after executing the tool and generating a response from the model.
        """
        user_prompt = build_prompt_structure(prompt=user_msg, role="user")

        tool_chat_history = ChatHistory(
            [
                build_prompt_structure(
                    prompt=TOOL_SYSTEM_PROMPT % self.add_tool_signatures(),
                    role="system",
                ),
                user_prompt,
            ]
        )
        agent_chat_history = ChatHistory([user_prompt])

        tool_call_response = completions_create(
            self.client, messages=tool_chat_history, model=self.model
        )
        tool_calls = extract_tag_content(str(tool_call_response), "tool_call")

        if tool_calls.found:
            observations = self.process_tool_calls(tool_calls.content)
            update_chat_history(
                agent_chat_history, f'f"Observation: {observations}"', "user"
            )

        return completions_create(self.client, agent_chat_history, self.model)


In [24]:
tool_agent = ToolAgent(tools=[hn_tool])

A quick check to see that everything works fine. If we ask the agent something unrelated to Hacker News, it shouldn't use the tool.

In [26]:
output = tool_agent.run(user_msg="Tell me your name")

KeyError: 'None'

In [ ]:
print(output)

Now, let's ask for specific information about Hacker News.

In [27]:
output = tool_agent.run(user_msg="Tell me the top 5 Hacker News stories right now")


Using Tool: fetch_top_hacker_news_stories

Tool call dict: 
{'name': 'fetch_top_hacker_news_stories', 'arguments': {'top_n': 5}, 'id': 1}

Tool result: 
[{"title": "DeepSeek: X2 Speed for WASM with SIMD", "url": "https://simonwillison.net/2025/Jan/27/llamacpp-pr/"}, {"title": "How has DeepSeek improved the Transformer architecture?", "url": "https://epoch.ai/gradient-updates/how-has-deepseek-improved-the-transformer-architecture"}, {"title": "New Speculative Attacks on Apple CPUs", "url": "https://predictors.fail/"}, {"title": "Using UV as Your Shebang Line", "url": "https://akrabat.com/using-uv-as-your-shebang-line/"}, {"title": "Maxima in the browser using Embedded Common Lisp on WASM", "url": "https://maxima-on-wasm.pages.dev/"}]


In [28]:
print(output)

Based on the provided data, here are the top 5 Hacker News stories right now:

1. **DeepSeek: X2 Speed for WASM with SIMD** - https://simonwillison.net/2025/Jan/27/llamacpp-pr/
2. **How has DeepSeek improved the Transformer architecture?** - https://epoch.ai/gradient-updates/how-has-deepseek-improved-the-transformer-architecture
3. **New Speculative Attacks on Apple CPUs** - https://predictors.fail/
4. **Using UV as Your Shebang Line** - https://akrabat.com/using-uv-as-your-shebang-line/
5. **Maxima in the browser using Embedded Common Lisp on WASM** - https://maxima-on-wasm.pages.dev/

Please note that the ranking is not explicitly provided in the data, so this is just an ordered list of the stories. If you need the actual ranking or scores, you would need to retrieve that information from the Hacker News API or website.


---
There you have it!! A fully functional Tool!! 🛠️